## LePersonFinder
Description: looks for people. 
Notes: The following code is for colab specific stuff, but see the below for non-colab specific paths. 

In [1]:
import os
import sys

isColab = "google.colab" in sys.modules

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

    student_colab_path = ("/content/drive/MyDrive"
        + "/Colab Notebooks")
    sys.path.append(student_colab_path)

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import numpy as np
import os
import json
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from tensorflow import keras
from keras.layers import Dense, Rescaling

In [4]:
## COLAB SPECIFIC PATHS 
data_folder_path = "/content/drive/MyDrive/Colab Notebooks/heridal_keras_retinanet_voc"
images_path = data_folder_path + "/JPEGImages"
annotations_path = data_folder_path + "/Annotations/JSON"
ImageSets = data_folder_path + "/ImageSets/Main"

In [3]:
## GITHUB PATHS 
data_folder_path = "data/heridal_keras_retinanet_voc"
images_path = data_folder_path + "/JPEGImages"
annotations_path = data_folder_path + "/Annotations/JSON"
ImageSets = data_folder_path + "/ImageSets/Main"

In [5]:
def xml_to_json(xml_folder, json_folder):
    os.makedirs(json_folder, exist_ok=True)

    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_folder, xml_file)
            json_path = os.path.join(json_folder, xml_file.replace('.xml', '.json'))

            tree = ET.parse(xml_path)
            root = tree.getroot()

            data_dict = {
                'folder': root.find('folder').text if root.find('folder') is not None else 'Unknown',
                'filename': root.find('filename').text if root.find('filename') is not None else 'Unknown',
                'size': {
                    'width': int(root.find('size/width').text) if root.find('size/width') is not None else 0,
                    'height': int(root.find('size/height').text) if root.find('size/height') is not None else 0,
                    'depth': int(root.find('size/depth').text) if root.find('size/depth') is not None else 0,
                },
                'objects': []
            }

            for obj in root.findall('object'):
                bbox = obj.find('bndbox')
                data_dict['objects'].append({
                    'name': obj.find('name').text if obj.find('name') is not None else 'Unknown',
                    'pose': obj.find('pose').text if obj.find('pose') is not None else 'Unspecified',
                    'truncated': int(obj.find('truncated').text) if obj.find('truncated') is not None else 0,
                    'difficult': int(obj.find('difficult').text) if obj.find('difficult') is not None else 0,
                    'bbox': {
                        'xmin': int(bbox.find('xmin').text) if bbox.find('xmin') is not None else 0,
                        'ymin': int(bbox.find('ymin').text) if bbox.find('ymin') is not None else 0,
                        'xmax': int(bbox.find('xmax').text) if bbox.find('xmax') is not None else 0,
                        'ymax': int(bbox.find('ymax').text) if bbox.find('ymax') is not None else 0,
                    }
                })

            with open(json_path, 'w') as json_file:
                json.dump(data_dict, json_file, indent=4)

In [6]:
#only run if json files don't exist yet
xml_folder = data_folder_path + "/Annotations"
json_folder = annotations_path
xml_to_json(xml_folder, json_folder)

In [7]:
def parse_json(json_string):
    data = json.loads(json_string.decode('utf-8'))
    if 'objects' not in data or not data['objects']:
        return np.zeros((0, 4), dtype=np.float32), np.int32(data['size']['width']), np.int32(data['size']['height'])
    bounding_boxes = np.array([
        [np.int32(obj['bbox']['xmin']), np.int32(obj['bbox']['ymin']),
         np.int32(obj['bbox']['xmax']), np.int32(obj['bbox']['ymax'])]
        for obj in data['objects'] if obj['name'] == 'person'
    ], dtype=np.float32)
    original_width = np.int32(data['size']['width'])
    original_height = np.int32(data['size']['height'])
    return bounding_boxes, original_width, original_height

def load_json(annotation_path):
    json_string = tf.io.read_file(annotation_path)
    bounding_boxes, original_width, original_height = tf.numpy_function(
        parse_json, [json_string], [tf.float32, tf.int32, tf.int32]
    )
    bounding_boxes.set_shape([None, 4])
    original_width.set_shape([])
    original_height.set_shape([])
    return bounding_boxes, original_width, original_height

In [8]:
IMAGE_SIZE = 256
PATCH_SIZE = 16
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2
PROJECTION_DIM = 64

In [9]:
def scale_bounding_boxes(bounding_boxes, original_width, original_height, new_width, new_height, pad_w, pad_h):
  if tf.size(bounding_boxes) == 0:
    return tf.zeros((0, 4), dtype=tf.float32)

  original_width = tf.cast(original_width, tf.float32)
  original_height = tf.cast(original_height, tf.float32)
  new_width = tf.cast(new_width, tf.float32)
  new_height = tf.cast(new_height, tf.float32)
  pad_w = tf.cast(pad_w[0], tf.float32)
  pad_h = tf.cast(pad_h[0], tf.float32)

  scale_x = new_width / original_width
  scale_y = new_height / original_height

  xmin_scaled = bounding_boxes[:, 0] * scale_x + pad_w
  ymin_scaled = bounding_boxes[:, 1] * scale_y + pad_h
  xmax_scaled = bounding_boxes[:, 2] * scale_x + pad_w
  ymax_scaled = bounding_boxes[:, 3] * scale_y + pad_h

  bounding_boxes_scaled = tf.stack([xmin_scaled, ymin_scaled, xmax_scaled, ymax_scaled], axis=1)

  return bounding_boxes_scaled

In [10]:
def compute_overlap_counts(bboxes, image_size, patch_size):
    num_patches_side = image_size // patch_size
    patch_indices = tf.range(0, image_size, patch_size)
    start_indices = tf.cast(tf.stack(tf.meshgrid(patch_indices, patch_indices, indexing='ij'), axis=-1), tf.float32)
    end_indices = start_indices + patch_size

    overlap_counts = tf.zeros((num_patches_side, num_patches_side), dtype=tf.int32)

    for bbox in bboxes:
        xmin, ymin, xmax, ymax = bbox[0], bbox[1], bbox[2], bbox[3]
        overlaps_x = tf.logical_and(start_indices[..., 1] < xmax, start_indices[..., 1] + patch_size > xmin)
        overlaps_y = tf.logical_and(start_indices[..., 0] < ymax, start_indices[..., 0] + patch_size > ymin)
        overlaps = tf.logical_and(overlaps_x, overlaps_y)

        overlap_counts += tf.cast(overlaps, tf.int32)
    
    return tf.expand_dims(overlap_counts, -1)

In [11]:
def preprocess_image(image_path, annotation_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)

  original_shape = tf.cast(tf.shape(image)[:2], tf.float32)
  ratio = IMAGE_SIZE / tf.reduce_max(original_shape)
  new_shape = tf.round(original_shape * ratio)

  image = tf.image.resize(image, tf.cast(new_shape, tf.int32))

  pad_w = (IMAGE_SIZE - new_shape[1]) / 2
  pad_h = (IMAGE_SIZE - new_shape[0]) / 2
  pad_w = [pad_w, IMAGE_SIZE - new_shape[1] - pad_w]
  pad_h = [pad_h, IMAGE_SIZE - new_shape[0] - pad_h]

  image = tf.pad(image, [[int(pad_h[0]), int(pad_h[1])], [int(pad_w[0]), int(pad_w[1])], [0, 0]], constant_values=0)
  image = Rescaling(1./255)(image)

  patches = tf.image.extract_patches(
      images=tf.expand_dims(image, 0),
      sizes=[1, PATCH_SIZE, PATCH_SIZE, 1],
      strides=[1, PATCH_SIZE, PATCH_SIZE, 1],
      rates=[1, 1, 1, 1],
      padding='VALID'
  )
  patches = tf.reshape(patches, [-1, PATCH_SIZE * PATCH_SIZE * 3])

  bounding_boxes, original_width, original_height = load_json(annotation_path)
  bounding_boxes_scaled = scale_bounding_boxes(
      bounding_boxes, original_width, original_height,
      new_shape[1], new_shape[0], pad_w, pad_h
  )

  overlap_counts = compute_overlap_counts(bounding_boxes_scaled, IMAGE_SIZE, PATCH_SIZE)

  return patches, overlap_counts, bounding_boxes_scaled

In [78]:
def file_exists(file_path):
    return tf.io.gfile.exists(file_path.numpy().decode())

def create_filtered_dataset(images_path, annotations_path, subset_prefix):
    image_files = tf.data.Dataset.list_files(os.path.join(images_path, subset_prefix + '_*.jpg'))

    def filter_func(image_file):
        annotation_file = tf.strings.regex_replace(image_file, 'JPEGImages', 'Annotations')
        annotation_file = tf.strings.regex_replace(annotation_file, '\.jpg', '.xml')
        return tf.py_function(file_exists, [annotation_file], Tout=tf.bool)

    filtered_image_dataset = image_files.filter(filter_func)

    def load_and_preprocess_image(image_file):
        annotation_file = tf.strings.regex_replace(image_file, 'JPEGImages', 'Annotations/JSON')
        annotation_file = tf.strings.regex_replace(annotation_file, '\.jpg', '.json')
        return preprocess_image(image_file, annotation_file)

    dataset = filtered_image_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [13]:
train_dataset = create_filtered_dataset(images_path, annotations_path, 'train')
test_dataset = create_filtered_dataset(images_path, annotations_path, 'test')

for patches, overlap_counts, bboxes in train_dataset.take(1):
    print('Train patches shape:', patches.shape)
    print('Train bounding boxes:', bboxes)
    print('Train overlap counts shape:', overlap_counts.shape)
    print('Train sample overlap counts:', overlap_counts.numpy().flatten()[:256])

for patches, overlap_counts, bboxes in test_dataset.take(1):
    print('Test patches shape:', patches.shape)
    print('Test bounding boxes:', bboxes)
    print('Test overlap counts shape:', overlap_counts.shape)
    print('Test sample overlap counts:', overlap_counts.numpy().flatten()[:256])

2024-04-20 11:36:21.995090: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1583]
	 [[{{node Placeholder/_0}}]]
2024-04-20 11:36:21.995524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1583]
	 [[{{node Placeholder/_0}}]]


Train patches shape: (256, 768)
Train bounding boxes: tf.Tensor(
[[222.848    108.22401  226.62401  113.600006]
 [ 23.04     143.42401   26.560001 146.176   ]
 [170.11201  212.92801  173.184    217.53601 ]], shape=(3, 4), dtype=float32)
Train overlap counts shape: (16, 16, 1)
Train sample overlap counts: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


2024-04-20 11:36:22.617993: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [101]
	 [[{{node Placeholder/_0}}]]
2024-04-20 11:36:22.618230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [101]
	 [[{{node Placeholder/_0}}]]


Test patches shape: (256, 768)
Test bounding boxes: tf.Tensor(
[[119.104004 140.09601  121.79201  145.728   ]
 [121.66401  141.632    125.12     147.58401 ]
 [125.184006 144.384    127.04001  147.45601 ]], shape=(3, 4), dtype=float32)
Test overlap counts shape: (16, 16, 1)
Test sample overlap counts: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
def display_image_with_bboxes(image_patches, overlap_counts, bbox_data, num_patches_side):
    patches_reshaped = tf.reshape(image_patches, [num_patches_side, num_patches_side, PATCH_SIZE, PATCH_SIZE, 3])
    patches_transposed = tf.transpose(patches_reshaped, [0, 2, 1, 3, 4])
    image_reconstructed = tf.reshape(patches_transposed, [IMAGE_SIZE, IMAGE_SIZE, 3])

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(image_reconstructed)

    for bbox in bbox_data:
        y_min = bbox[1]
        x_min = bbox[0]
        y_max = bbox[3]
        x_max = bbox[2]
        print(f"Bounding Box: [{x_min}, {y_min}, {x_max}, {y_max}]")

        width = x_max - x_min
        height = y_max - y_min

        rect = mpatches.Rectangle((x_min, y_min), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    for i in range(num_patches_side):
        for j in range(num_patches_side):
            if overlap_counts[i, j, 0] > 0:
                patch_x = j * PATCH_SIZE
                patch_y = i * PATCH_SIZE
                rect = mpatches.Rectangle((patch_x, patch_y), PATCH_SIZE, PATCH_SIZE, linewidth=1, edgecolor='yellow', facecolor='none', linestyle='--')
                ax.add_patch(rect)
                ax.text(patch_x + PATCH_SIZE / 2, patch_y + PATCH_SIZE / 2, str(overlap_counts[i, j, 0]),
                        color='yellow', ha='center', va='center', fontsize=8)

    plt.title("Reconstructed Image with Bounding Boxes and Highlight Patches")
    plt.axis('off')
    plt.show()

for image_patches, overlap_counts, bbox_data in train_dataset.take(1):
    num_patches_side = IMAGE_SIZE // PATCH_SIZE
    display_image_with_bboxes(image_patches.numpy(), overlap_counts.numpy(), bbox_data.numpy(), num_patches_side)

## Model Class
The following code is for the Model class of the Transformer model. 

Some key notes about our dataset: 
- 7 classes 
- (256, 768) inputs

Some key notes about other parameters: 
- Query: of shape (batch_sz, target_seq_length, feature_dim)
- Value: of shape (batch_sz, source_seq_length, feature_dim)
- Key: of shape (batch_sz, source_seq_length, feature_dim)

Our architecture using the below classes goes as follows: 


### Hyperparameters/Stats
Hyperparameters and stats for our model pre-compilation.

In [90]:
## Hyperparameters & general stats
params = {
    'batch_sz': 64, 
    'num_classes': 8,
    'vocab_sz': 8,  
    'num_att_heads': 1, 
    'key_dim': 10, 
    'query_dim': 10, 
    'value_dim': 10, 
    'output_sz': 8, 
    'window_sz': 256,
    'num_features': 768, 
    'emb_sz': 132, 
    'num_layers': 2, 
    'learning_rate': 0.01, 
    'num_epochs': 20, 
}

In [ ]:
## TODO: This takes like way too long to run - could anyone create tensors that have a batch dimension?

all_cts = set()
all_cts.add(0)
img_patches_shape = set()
overlap_cts_shape = set()
bbox_data_shape = set() 
len_train = 0

train_img_patches = tf.Variable(tf.random.normal((1546, 256, 768), dtype=tf.float32))
train_overlap_counts = tf.Variable(tf.zeros((1546, 16, 16, 1), dtype=tf.int32))

for image_patches, overlap_counts, bbox_data in train_dataset:
    # train_img_patches[len_train].assign(image_patches)
    # train_overlap_counts[len_train].assign(overlap_counts)
    len_train += 1  
    uq_ct = np.unique(overlap_counts.numpy())
    uq_ct = uq_ct[1:] 
    img_patches_shape.add(image_patches.shape)
    overlap_cts_shape.add(overlap_counts.shape)
    bbox_data_shape.add(bbox_data.shape) ## Bbox shape is NOT uniform unfortunately 

    for el in uq_ct: 
        all_cts.add(el)

print("This is the total number of classes: ", all_cts)
print("This is the size of the train dataset: ", len_train)
print("These are the img patches shape: ", img_patches_shape)
print("These are overlap cts shape: ", overlap_cts_shape)
print("These are bbox shape: ", bbox_data_shape)

len_test = 0
test_img_patches = tf.Variable(tf.random.normal((101, 256, 768), dtype=tf.float32))
test_overlap_counts = tf.Variable(tf.zeros((101, 16, 16, 1), dtype=tf.int32))
for test_image_patches, test_overlap_counts, test_bbox_data in test_dataset:
    # test_img_patches[len_test].assign(test_image_patches)
    # test_overlap_counts[len_test].assign(test_overlap_counts)
    len_test += 1

print("This is the size of the test dataset: ", len_test)

### Positional Encoding 
This defines our positional encoding class. This will positionally encode our input tensor so that the Transformer has some understanding of how the pixels are distributed spatially. 

In [147]:
## Taken from class - positional encoding method. 
def positional_encoding(length, depth):
  depth = depth/2
  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)
  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 
  return tf.cast(pos_encoding, dtype=tf.float32)

## Taken from class - positional embedding class that will look a token's embedding vector and add the corresponding position vector 
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, emb_sz):
    super().__init__()
    self.emb_sz = emb_sz
    # self.embedding = tf.keras.layers.Embedding(vocab_size, emb_sz, mask_zero=True) <- embedding for NLP-like tasks 
    self.embedding = tf.keras.layers.Dense(emb_sz)
    self.pos_encoding = positional_encoding(length=2048, depth=emb_sz)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.emb_sz, tf.float32)) # This factor sets the relative scale of the embedding and positonal_encoding.
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x
  
## UNIT TEST: check sizes @ embedding 
test_emb = tf.zeros((params['batch_sz'], params['window_sz'], params['num_features']))
embed_img = PositionalEmbedding(vocab_size=params['vocab_sz'], emb_sz=params['emb_sz'])
embed = embed_img(test_emb)
print("This is the positional embedding shape: ", embed.shape)


This is the positional embedding shape:  (64, 256, 132)


### Attention
- BaseAttention: super class that other variations of Attention can inherit from. Essentially combines Attention + LayerNormalization into one class.
- Other subclasses...

In [148]:
## Base Attention Class
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

## This class is just a basic implementation of self-attention 
class GlobalSelfAttention(BaseAttention): 
    def call(self, x): 
      attn_output, attn_scores = self.mha(query=x, key=x, value=x, return_attention_scores=True)
      # Cache the attention scores for plotting later.
      self.last_attn_scores = attn_scores
      x = self.add([x, attn_output])
      x = self.layernorm(x)
      return x
    
## This class is just the masked implementation of self-attention
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

## This class connects the Encoder and Decoder: takes in context sequence for keys and values as opposed to the input sequence.  
class CrossAttention(BaseAttention):
  def call(self, x, context):
    # Multi-headed attention 
    attn_output, attn_scores = self.mha(query=x, key=context, value=context, return_attention_scores=True)
    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x
  
sample_sa = GlobalSelfAttention(num_heads=params['num_att_heads'], key_dim=params['key_dim'])
att_on_emb = sample_sa(embed)
print("This is the shape post-self attention: ", att_on_emb.shape)

This is the shape post-self attention:  (64, 256, 132)


### FeedForward Layers
- This class defines feedforward layers that produce our desired output after we apply attention to our input. This combines ADD + LayerNormalization in as well. 


In [149]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, emb_sz, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(emb_sz),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x

### Encoder Block
There are two classes in this section: 
- Encoder Layer: one unit of the block. Attention -> LayerNorm -> FeedForward NN. 
- Encoder Block: we can have many stacked Encoder layers. All this does is create a positional embedding for the input, add it to the input, then pass that into a stack of Encoder layers. 

In [150]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, emb_sz, num_heads, dff, dropout_rate=0.1):
    super().__init__()
    self.self_attention = BaseAttention(
        num_heads=num_heads,
        key_dim=emb_sz,
        dropout=dropout_rate)
    self.ffn = FeedForward(emb_sz, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [151]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, emb_sz, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()
    self.emb_sz = emb_sz
    self.num_layers = num_layers
    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, emb_sz=emb_sz)
    self.enc_layers = [
        EncoderLayer(emb_sz=emb_sz,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Should result in shape of (batch_size, seq_len, emb_sz).
    x = self.dropout(x)  # Add dropout.
    for i in range(self.num_layers):
      x = self.enc_layers[i](x)
    return x  # Should result in shape of (batch_size, seq_len, emb_sz).

### Decoder Block 
There are two classes in this section: 
- Decoder Block: one unit of the block. Self attention -> Cross attention -> Feed forward network. 
- Decoder: stacked Decoder layers.

In [156]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, emb_sz, num_heads, dff, dropout_rate=0.1):
    super(DecoderLayer, self).__init__()
    ## Basic self-attention (tutorials might use causal self attention, look into what that does)
    self.causal_self_attention = CausalSelfAttention(num_heads=num_heads, key_dim=emb_sz, dropout=dropout_rate)
    ## Encoder-decoder attention 
    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=emb_sz,
        dropout=dropout_rate)
    self.ffn = FeedForward(emb_sz, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)
    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores
    x = self.ffn(x)  # Shape `(batch_size, seq_len, emb_sz)`.
    return x

In [153]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, emb_sz, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()
    self.emb_sz = emb_sz
    self.num_layers = num_layers
    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size, emb_sz=emb_sz)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [DecoderLayer(emb_sz=emb_sz, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate) for _ in range(num_layers)]
    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, emb_sz)
    x = self.dropout(x)
    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)
    self.last_attn_scores = self.dec_layers[-1].last_attn_scores
    return x # The shape of x is (batch_size, target_seq_len, emb_sz).

### Transformer Model
This combines our Encoder and Decoder blocks into a coherent model. It should look something like the following: 
- Positional Encoding + Addition of Context => 
- => Encoder Block: 
    - Encoder Layer 1: 
        - Self Attention 
        - Feed Forward NN 
            - Dense 1 
            - Dense 2 
            - Dropout 
            - Add & LayerNormalization
    - Encoder Layer 2: 
        - Self Attention 
        - Feed Forward NN 
            - Dense 1 
            - Dense 2 
            - Dropout 
            - Add & LayerNormalization 
    - etc ...
- => Decoder Block: 
    - Decoder Layer 1: 
        - Causal Self Attention 
        - Cross Attention 
        - Feed Forward NN 
            - Dense 1
            - Dense 2 
            - Dropout 
            - Add & LayerNormalization
    - etc ...

In [159]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, emb_sz, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.emb_context = tf.keras.layers.Dense(emb_sz)
    self.emb_patches = tf.keras.layers.Dense(emb_sz)
    
    self.encoder = Encoder(num_layers=num_layers, emb_sz=emb_sz,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, emb_sz=emb_sz,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  ## FORWARD PASS
  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the first argument.
    patches, bbox_context = inputs  

    # flatten & upsample the context, input: the following should return (batch_sz, 256, emb_sz)
    bbox_context = tf.reshape(bbox_context, (params['batch_sz'], -1, 1))
    bbox_context = self.emb_context(bbox_context)
    patches = self.emb_patches(patches)
    embedded = bbox_context + patches

    # The following should return: (batch_sz, 256, emb_sz)
    context = self.encoder(embedded) # ** (batch_size, context_len, emb_sz)

    # The following should return: (batch_size, 256, emb_sz)
    x = self.decoder(patches, context)  # ** (batch_size, target_len, emb_sz)

    # The following should return: (batch_size, 256, 8)
    logits = self.final_layer(x)  # ** (batch_size, target_len, target_vocab_size)

    # nk - idrk what this following section does...
    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics. b/250038731
      del logits._keras_mask
    except AttributeError:
      pass
    # Return the final output and the attention weights.
    return logits
  
  ## TRAIN STEP (1 per epoch)
  def train(): 
    pass 

### Accuracy and Loss Metrics
Title. 

In [131]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)
  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask
  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss

def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred
  mask = label != 0
  match = match & mask
  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

### Train and Fit the Model


In [162]:
## Instantiations of above objects
transformer = Transformer(num_layers=params['num_layers'], 
                          emb_sz=params['emb_sz'],
                          num_heads=params['num_att_heads'], 
                          dff=params['num_features'], 
                          input_vocab_size=params['vocab_sz'],
                          target_vocab_size=params['vocab_sz'])
optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])

## Compile the model 
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

## Custom train model method  
     # - notably can't just batch like below because bbox data is incorporated into the iterator, it isn't uniform size 
        #$ batched_train_dataset = train_dataset.batch(batch_size=params['batch_sz'], drop_remainder=False)
    # - had to modify Sam's stuff a little bit so that the last bbox thing was out the picture

def remove_bbox(c1, c2, c3):
    return c1, c2
batchable_train_data = train_dataset.map(remove_bbox)
batched_train_dataset = batchable_train_data.batch(batch_size=params['batch_sz'], drop_remainder=False)
batchable_test_data = test_dataset.map(remove_bbox)
batched_test_dataset = batchable_test_data.batch(batch_size=params['batch_sz'], drop_remainder=False)

def train(params, train_dataset): 
    for e in range(params['num_epochs']): 
        num_batches = len_train // params['batch_sz']
        train_dataset.shuffle(buffer_size=3) # shuffle the dataset every epoch
        for batch_number, batch in enumerate(batched_train_dataset): 
            patches, bbox_context = batch
            logits = transformer((patches, bbox_context))

train(params, train_dataset)

2024-04-20 19:15:40.989616: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1583]
	 [[{{node Placeholder/_0}}]]
2024-04-20 19:15:40.989879: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1583]
	 [[{{node Placeholder/_0}}]]


(64, 256, 8)
(64, 256, 8)
(64, 256, 8)


2024-04-20 19:15:49.162549: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): CANCELLED: Loop execution was cancelled.
	 [[{{node while/LoopCond/_30}}]]
2024-04-20 19:15:49.165635: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): CANCELLED: Loop execution was cancelled.
	 [[{{node while/LoopCond/_30}}]]
2024-04-20 19:15:49.192855: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): CANCELLED: Loop execution was cancelled.
	 [[{{node while/LoopCond/_30}}]]
2024-04-20 19:15:49.216771: I tensorflow/core/common_runtime/executor.cc:1197] [/job:

KeyboardInterrupt: 